In [1]:
import torchvision
import torch
import numpy as np
import matplotlib.pyplot as plt
import inspect

from torchsummary import summary
from torchvision import transforms, datasets

import time
from torch.utils.tensorboard import SummaryWriter
from ignite.handlers import ModelCheckpoint, EarlyStopping

from torch.utils.data import Subset, DataLoader
import sklearn.model_selection

from ignite.engine import Engine, Events
from ignite.metrics import Loss, Accuracy

from torchvision import models

torch.set_num_threads(8)

In [3]:
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])
valid_transforms = transforms.Compose([transforms.Resize(255),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

train_dataset = datasets.ImageFolder('gestos/train', transform=train_transforms)
valid_dataset = datasets.ImageFolder('gestos/valid', transform=valid_transforms)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=6)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [4]:
model = models.resnet18(pretrained=True, progress=True)
model.eval()

for param in model.parameters():
    param.requires_grad = False

neuronas = model.fc.in_features   #512
model.fc = torch.nn.Sequential(
    torch.nn.Linear(neuronas, 128),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(128,4),
    torch.nn.Softmax(dim=1)
)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


max_epochs = 100
device = torch.device('cpu')


def train_one_step(engine, batch):
    model.train()
    optimizer.zero_grad()
    x, y = batch
    x, y = x.to(device), y.to(device)
    yhat = model.forward(x)
    loss = criterion(yhat, y)
    loss.backward()
    optimizer.step()
    return loss.item() # Este output puede llamar luego como trainer.state.output


def evaluate_one_step(engine, batch):
    model.eval()
    with torch.no_grad():
        x, y = batch
        x, y = x.to(device), y.to(device)
        yhat = model.forward(x)
        loss = criterion(yhat, y)
        return yhat, y
    

trainer = Engine(train_one_step)
evaluator = Engine(evaluate_one_step)
metrics = {'Loss': Loss(criterion), 'Acc': Accuracy()}
for name, metric in metrics.items():
    metric.attach(evaluator, name)

In [ ]:
with SummaryWriter(log_dir=f'/tmp/tensorboard/prerun' + runtime) as writer:
    
    @trainer.on(Events.EPOCH_COMPLETED(every=1)) # Cada 1 epocas
    def log_results(engine):
        # Evaluo el conjunto de entrenamiento
        evaluator.run(train_loader) 
        writer.add_scalar("train/loss", evaluator.state.metrics['Loss'], engine.state.epoch)
        writer.add_scalar("train/accy", evaluator.state.metrics['Acc'], engine.state.epoch)
        
        evaluator.run(valid_loader) 
        writer.add_scalar("valid/loss", evaluator.state.metrics['Loss'], engine.state.epoch)
        writer.add_scalar("valid/accy", evaluator.state.metrics['Acc'], engine.state.epoch)
        #print("Época: ", engine.state.epoch,"   Demora: ", time.time()-seconds, "[seg]    ", "Loss: ", evaluator.state.output[2])
    # Guardo el mejor modelo en validación
    best_model_handler = ModelCheckpoint(dirname=folder+runtime+'/bests', require_empty=False, filename_prefix="best", n_saved=5,
                                         score_function=lambda engine: -engine.state.metrics['Loss'],
                                         score_name="val_loss")
    # Lo siguiente se ejecuta cada ves que termine el loop de validación
    evaluator.add_event_handler(Events.COMPLETED, 
                                best_model_handler, {'ResNet18': model})
    trainer.run(train_loader, max_epochs=max_epochs)

In [ ]:
test_transforms = transforms.Compose([transforms.Resize(255),
                                       transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])
test_data = datasets.ImageFolder('gestos/test', transform=test_transforms)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=256, shuffle=False)

In [ ]:
model.eval()
model.load_state_dict(torch.load(folder+runtime+'bests/'+file, map_location=torch.device('cpu')))

test_loader = DataLoader(test_data, shuffle=False, batch_size=256)

test_targets = np.array(test_data.targets)
prediction_test = []


for mbdata, label in test_loader:

    logits = model.forward(mbdata)
    prediction_test.append(logits.argmax(dim=1).detach().numpy())
prediction_test = np.concatenate(prediction_test)

from sklearn.metrics import confusion_matrix, classification_report

cm.append(confusion_matrix(test_targets, prediction_test))
display(cm)
report = classification_report(test_targets, prediction_test)